In [1]:
import os, sys
sys.path += ["../.."]
sys.path += ["../../.."]
import torch
import torch.functional as F
from applications.DECA.interactive_deca_decoder import load_deca_and_data, test, plot_results
from applications.DECA.optimize_latent_space import TargetEmotionCriterion, optimize
import copy
from gdl.layers.losses.EmoNetLoss import EmoNetLoss
from gdl.models.DECA import DecaModule, DECA, DecaMode
from skimage.io import imread
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [3]:
target_image = "~/Workspace/mount/scratch/rdanecek/data/aff-wild2/processed/processed_2021_Jan_19_20-25-10/VA_Set/" \
               "detections/Train_Set/82-25-854x480/002400_000.png"
loss_to_use = TargetEmotionCriterion(target_image)
path_to_models = '/home/rdanecek/Workspace/mount/scratch/rdanecek/emoca/finetune_deca'
# run_name = '2021_03_01_11-31-57_VA_Set_videos_Train_Set_119-30-848x480.mp4_EmoNetLossB_F1F2VAECw-0.00150_CoSegmentGT_DeSegmentRend'
run_name = '2021_03_08_22-30-55_VA_Set_videos_Train_Set_119-30-848x480.mp4CoPhotoCoLMK_IDW-0.15_Aug_early'
stage = 'detail'
relative_to_path = '/ps/scratch/'
replace_root_path = '/home/rdanecek/Workspace/mount/scratch/'
deca, dm = load_deca_and_data(path_to_models, run_name, stage, relative_to_path, replace_root_path)
deca.deca.config.train_coarse = True
deca.deca.config.mode = DecaMode.DETAIL
# deca.deca.config.mode = DecaMode.COARSE
image_index = 390 * 4 + 1
values, visdict = test(deca, dm, image_index)


Loading the EmoNet model from /home/rdanecek/Workspace/Repos/emonet/pretrained/emonet_8.pth.


/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


Looking for checkpoint in '/home/rdanecek/Workspace/mount/scratch/rdanecek/emoca/finetune_deca/2021_03_08_22-30-55_VA_Set_videos_Train_Set_119-30-848x480.mp4CoPhotoCoLMK_IDW-0.15_Aug_early/detail/checkpoints'
Found 4 checkpoints
 - /home/rdanecek/Workspace/mount/scratch/rdanecek/emoca/finetune_deca/2021_03_08_22-30-55_VA_Set_videos_Train_Set_119-30-848x480.mp4CoPhotoCoLMK_IDW-0.15_Aug_early/detail/checkpoints/deca-epoch=14-val_loss=0.9390.ckpt
 - /home/rdanecek/Workspace/mount/scratch/rdanecek/emoca/finetune_deca/2021_03_08_22-30-55_VA_Set_videos_Train_Set_119-30-848x480.mp4CoPhotoCoLMK_IDW-0.15_Aug_early/detail/checkpoints/deca-epoch=17-val_loss=0.9394.ckpt
 - /home/rdanecek/Workspace/mount/scratch/rdanecek/emoca/finetune_deca/2021_03_08_22-30-55_VA_Set_videos_Train_Set_119-30-848x480.mp4CoPhotoCoLMK_IDW-0.15_Aug_early/detail/checkpoints/deca-epoch=19-val_loss=0.9407.ckpt
 - /home/rdanecek/Workspace/mount/scratch/rdanecek/emoca/finetune_deca/2021_03_08_22-30-55_VA_Set_videos_Train_Set

../../models/DecaFLAME.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(lmk_embeddings['dynamic_lmk_faces_idx'], dtype=torch.long))
../../models/DecaFLAME.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(lmk_embeddings['dynamic_lmk_bary_coords'], dtype=self.dtype))


fc.weight  not available in reconstructed resnet
fc.bias  not available in reconstructed resnet
copy resnet state dict finished!


/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/pytorch3d/io/obj_io.py:457: UserWarning: Mtl file does not exist: /home/rdanecek/Workspace/mount/scratch/yfeng/Data/FLAME/geometry/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


Loading the EmoNet model from /home/rdanecek/Workspace/Repos/emonet/pretrained/emonet_8.pth.
DECA MODE RECONFIGURED TO: DecaMode.DETAIL
DECA loaded
The data will be loaded from: '/home/rdanecek/Workspace/mount/scratch/rdanecek/data/aff-wild2'
The dataset is already processed. Loading
Loading metadata of FaceVideoDataset from: '/home/rdanecek/Workspace/mount/scratch/rdanecek/data/aff-wild2/processed/processed_2021_Jan_19_20-25-10/metadata.pkl'
Loading the dataset
FaceVideoDataset already loaded.
Dataset loaded
Looking for video 148 in 477
The dataset is already processed. Loading
FaceVideoDataset already loaded.
Loading the dataset
FaceVideoDataset already loaded.
Dataset loaded
Found processed filelists in '/home/rdanecek/Workspace/mount/scratch/rdanecek/data/aff-wild2/processed/processed_2021_Jan_19_20-25-10/cache/47c56b8af9dcf6b425af013417cc6450'. Reprocessing will not be needed. Loading ...
Loading done
Dataset split found in: '/home/rdanecek/Workspace/mount/scratch/rdanecek/data/af

/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


In [2]:
print(values.keys())
plot_results(visdict, "title")
optimize(deca, values, loss_to_use=loss_to_use)

Looking for checkpoint in '/home/rdanecek/Workspace/mount/scratch/rdanecek/emoca/finetune_deca/2021_03_01_11-31-57_VA_Set_videos_Train_Set_119-30-848x480.mp4_EmoNetLossB_F1F2VAECw-0.00150_CoSegmentGT_DeSegmentRend/coarse/checkpoints'
Found 3 checkpoints
 - /home/rdanecek/Workspace/mount/scratch/rdanecek/emoca/finetune_deca/2021_03_01_11-31-57_VA_Set_videos_Train_Set_119-30-848x480.mp4_EmoNetLossB_F1F2VAECw-0.00150_CoSegmentGT_DeSegmentRend/coarse/checkpoints/deca-epoch=03-val_loss=0.18.ckpt
 - /home/rdanecek/Workspace/mount/scratch/rdanecek/emoca/finetune_deca/2021_03_01_11-31-57_VA_Set_videos_Train_Set_119-30-848x480.mp4_EmoNetLossB_F1F2VAECw-0.00150_CoSegmentGT_DeSegmentRend/coarse/checkpoints/deca-epoch=04-val_loss=0.17.ckpt
 - /home/rdanecek/Workspace/mount/scratch/rdanecek/emoca/finetune_deca/2021_03_01_11-31-57_VA_Set_videos_Train_Set_119-30-848x480.mp4_EmoNetLossB_F1F2VAECw-0.00150_CoSegmentGT_DeSegmentRend/coarse/checkpoints/deca-epoch=09-val_loss=0.19.ckpt
Loading checkpoint '

../../models/DecaFLAME.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(lmk_embeddings['dynamic_lmk_faces_idx'], dtype=torch.long))
../../models/DecaFLAME.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(lmk_embeddings['dynamic_lmk_bary_coords'], dtype=self.dtype))


fc.weight  not available in reconstructed resnet
fc.bias  not available in reconstructed resnet
copy resnet state dict finished!
Start training from scratch


/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/pytorch3d/io/obj_io.py:457: UserWarning: Mtl file does not exist: /home/rdanecek/Workspace/mount/scratch/yfeng/Data/FLAME/geometry/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


Adding EmoNet path '/home/rdanecek/Workspace/Repos/emonet'
Loading the EmoNet model from /home/rdanecek/Workspace/Repos/emonet/pretrained/emonet_8.pth.
Setting E_flame to eval
Setting E_detail to eval
Setting D_detail to eval
DECA MODE RECONFIGURED TO: DecaMode.COARSE
The data will be loaded from: '/home/rdanecek/Workspace/mount/scratch/rdanecek/data/aff-wild2'
The dataset is already processed. Loading
Loading metadata of FaceVideoDataset from: '/home/rdanecek/Workspace/mount/scratch/rdanecek/data/aff-wild2/processed/processed_2021_Jan_19_20-25-10/metadata.pkl'
Loading the dataset
FaceVideoDataset already loaded.
Dataset loaded
Looking for video 148 in 477
The dataset is already processed. Loading
FaceVideoDataset already loaded.
Loading the dataset
FaceVideoDataset already loaded.
Dataset loaded
Cached dataset not found in /home/rdanecek/Workspace/mount/scratch/rdanecek/data/aff-wild2/processed/processed_2021_Jan_19_20-25-10/cache/2d069c50223452b788c7c14f08e8a2184fb768f10c96da284deae4

In [3]:
# %matplotlib notebook
%matplotlib inline
import matplotlib 
matplotlib.interactive(False)
matplotlib.rcParams['figure.figsize'] = [20, 4]
# wx, gtk, osx, tk, empty uses default
# %matplotlib qt
# %matplotlib notebook

In [7]:
image_index = 390*4 # big nose frown fail 
# image_index = 75*4
# image_index = 9*4 +3
values_fail, visdict_fail = test(deca, dm, image_index)

/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [12]:
#plot_results(visdict_fail, "Fail", plot_mode)
values_fail['cam']

tensor([[1.0523e+01, 5.2308e-03, 2.5351e-02]], device='cuda:0')

In [36]:
image_index = 390*4 - 1
values_before_fail, visdict_before_fail = test(deca, dm, image_index)

/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


In [9]:
## FUNCTIONING IMAGE 
image_index = 0
values_success, visdict_success = test(deca, dm, image_index)
values_success['cam']

/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


tensor([[1.0696e+01, 5.2629e-03, 5.4284e-02]], device='cuda:0')

In [13]:
plot_results(visdict_success, "Success")
values_success['cam']

SyntaxError: invalid syntax (<ipython-input-13-1260470546f5>, line 2)

In [41]:
from copy import deepcopy 
values_replaced = deepcopy(values_fail)
values_replaced['cam'] = deepcopy(values_before_fail['cam'])
values_replaced, visdict_replaced = test(deca, dm, values=values_replaced)

/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


In [14]:
plot_results(visdict_replaced, "Fail with successful camera")
values_replaced['cam']

NameError: name 'values_replaced' is not defined

In [43]:
from copy import deepcopy 
values_replaced2 = deepcopy(values_before_fail)
values_replaced2['cam'] = deepcopy(values_fail['cam'])

values_replaced2, visdict_replaced2 = test(deca, dm, values=values_replaced2)

/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/rdanecek/anaconda3/envs/work/lib/python3.6/site-packages/torch/nn/functional.py:3385: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn("Default grid_sample and affine_grid behavior has changed "


In [15]:
#plot_results(visdict_replaced2,  "Success with failed camera")
values_replaced2['cam']

NameError: name 'values_replaced2' is not defined

In [16]:
#plot_results(visdict_success, "Success example")
#plot_results(visdict_before_fail, "Successful frame before fail")
#plot_results(visdict_fail, "Fail")
#plot_results(visdict_replaced, "Fail with successful camera")
#plot_results(visdict_replaced2, "Success with failed camera")